# TODO
- try different wavelets and classifier parameters
- stacking classifier?
- sample from classes such that they all have same cardinality
- try with CNN (or RNN)
- hiearchical classification: first classify 3 vs all, then 0 vs 1 vs 2

# Imports & setup

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from feature_extraction import *
import scipy
from scipy import fft
from scipy import signal
from collections import Counter
import pywt
from biosppy.signals import ecg
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

In [2]:
n_cores = 1
random_state = 71
sampling_rate = 300
data_directory = 'data/'

waveletname = 'db31'
level = 5 #10

# Data import

In [3]:
X_train_df = pd.read_csv(data_directory + 'X_train.csv', index_col='id')
y_train_df = pd.read_csv(data_directory + 'y_train.csv', index_col='id')
X_test_df = pd.read_csv(data_directory + 'X_test.csv', index_col='id')

# Length adjustments

### Drop trailing NaN

In [4]:
def drop_trailing_na(df: pd.DataFrame):
    return [df.loc[i].dropna().to_numpy() for i in range(df.shape[0])]

X_train_full = drop_trailing_na(X_train_df)
y_train_full = y_train_df['y'].to_numpy()
X_test = drop_trailing_na(X_test_df)

# Noise handling

In [5]:
def wavelet_transform(signal):
    return pywt.wavedec(signal, waveletname, level=level)

def wavelet_noise_cancellation(signal):
    coeffs = wavelet_transform(signal)
    return pywt.waverec(coeffs, waveletname)

def wavelet_noise_cancellation_bulk(data):
    result = []
    for signal in data:
        result.append(wavelet_noise_cancellation(signal))
    return result

In [6]:
"""X_train_full_filtered = wavelet_noise_cancellation_bulk(X_train_full)
X_test_filtered = wavelet_noise_cancellation_bulk(X_test)"""

'X_train_full_filtered = wavelet_noise_cancellation_bulk(X_train_full)\nX_test_filtered = wavelet_noise_cancellation_bulk(X_test)'

# Separation in training and validation

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=random_state)

# Features extraction

### Features to extract

In [14]:
def calculate_entropy(list_values):
    value, probabilities = np.unique(list_values, return_counts=True)
    """counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]"""
    entropy = scipy.stats.entropy(probabilities)
    return [entropy]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]
 
def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def get_array_features(arr):
    features = []
    features += calculate_entropy(arr)
    features += calculate_crossings(arr)
    features += calculate_statistics(arr)
    return features

def get_wavelet_features(signal):
    features = []
    list_coeff = wavelet_transform(signal)
    for coeff in list_coeff:
        features += get_array_features(coeff)
    return features

def to_array(x):
    return np.argwhere(x == 1).T[0]

def calculate_consecutive_diff(x):
    return np.ediff1d(x)

def calculate_diff_old(x, y):
    res = []
    x_index, y_index = 0, 0
    while x_index < len(x) and y_index < len(y):
        if x_index < len(x) - 1 and x[x_index + 1] < y[y_index]:
            x_index += 1
        elif x[x_index] > y[y_index]:
            y_index += 1
        else:
            res.append(y[y_index] - x[x_index])
            x_index += 1
            y_index += 1
    return np.array(res)

def calculate_diff(x, y):
    res = []
    for i in range(len(x)):
        res.append(y[i] - x[i])
    return np.array(res)

def get_ecg_values_old(data):
    result = ecg.ecg(data, sampling_rate=sampling_rate, show=False)

    templates = result['templates']

    p_peaks, q_peaks, r_peaks, s_peaks, t_peaks = [], [], [], [], []
    p_values, q_values, r_values, s_values, t_values = [], [], [], [], []
    pr_diff, ps_diff, pt_diff, qs_diff, qt_diff, rt_diff = [], [], [], [], [], []
    # extract the pqrst indexes for each template (note that the indexes are sampled at 300Hz!)
    for template in templates:
        # calculate the locations
        try:
            # get local maximas and minimas with signal library
            loc_max = np.array(signal.argrelextrema(template, np.greater))
            loc_min = np.array(signal.argrelextrema(template, np.less))

            # find the maximum, but cut the search area to the first half to avoid
            # finding peaks at the wrong place
            r = np.argmax(template[: int(len(template) / 2)])
            # q and s are the first minima after and before the r value
            q = loc_min[loc_min < r][-1]
            s = loc_min[loc_min > r][0]
            # p and t are the first maxima after and before the r value
            p = loc_max[loc_max < r][-1]
            t = loc_max[loc_max > r][0]

            p_peaks.append(p)
            p_values.append(template[p])
            q_peaks.append(q)
            q_values.append(template[q])
            r_peaks.append(r)
            r_values.append(template[r])
            s_peaks.append(s)
            s_values.append(template[s])
            t_peaks.append(t)
            t_values.append(template[t])
            # calculate different values
            pr_diff.append(r - p)
            ps_diff.append(s - p)
            pt_diff.append(t - p)
            qs_diff.append(s - q)
            qt_diff.append(t - q)
            rt_diff.append(t - r)
        except:
            pr_diff.append(0)
            ps_diff.append(0)
            pt_diff.append(0)
            qs_diff.append(0)
            qt_diff.append(0)
            rt_diff.append(0)

    beats = fft.fft(templates)
    heart_rate = sampling_rate * (60.0 / np.diff(result['rpeaks']))
    heart_rate = np.append(heart_rate, heart_rate[-1]).reshape(-1, 1)
    
    """RRinterval = calculate_consecutive_diff(r_peaks)
    PPinterval = calculate_consecutive_diff(p_peaks)"""

    """Pduration = calculate_diff(ponsets, poffsets)
    PRsegment = calculate_diff(poffsets, q_peaks)
    PRinterval = calculate_diff(ponsets, q_peaks)
    PRsegment = calculate_diff(poffsets, q_peaks)
    QRScomplex = calculate_diff(q_peaks, s_peaks)
    QTinterval = calculate_diff(q_peaks, toffsets)
    STsegment = calculate_diff(s_peaks, tonsets)
    STTsegment = calculate_diff(s_peaks, toffsets)
    TPinterval = calculate_diff(toffsets, ponsets)"""

    #return heart_rate, RRinterval, PPinterval, np.real(beats), np.imag(beats), \
    return heart_rate, np.real(beats), np.imag(beats), \
        np.array(p_values), np.array(q_values), np.array(r_values), np.array(s_values), np.array(t_values), \
        np.array(pr_diff), np.array(ps_diff), np.array(pt_diff), np.array(qs_diff), np.array(qt_diff), np.array(rt_diff)

    """p_vals.append(template[p])
    q_vals.append(template[q])
    r_vals.append(template[r])
    s_vals.append(template[s])
    t_vals.append(template[t])"""

    """rpeaks_diff = np.ediff1d(rpeaks)
    return rpeaks_diff, templates"""
    
    signals, rpeaks = nk.ecg_process(data, sampling_rate=sampling_rate)

    heart_rate = np.array(signals['ECG_Rate'])

    ppeaks = to_array(signals['ECG_P_Peaks'])
    ponsets = to_array(signals['ECG_P_Onsets'])
    poffsets = to_array(signals['ECG_P_Offsets'])
    qpeaks = to_array(signals['ECG_Q_Peaks'])
    rpeaks = to_array(signals['ECG_R_Peaks'])
    ronsets = to_array(signals['ECG_R_Onsets'])
    roffsets = to_array(signals['ECG_R_Offsets'])
    speaks = to_array(signals['ECG_S_Peaks'])
    tpeaks = to_array(signals['ECG_T_Peaks'])
    tonsets = to_array(signals['ECG_T_Onsets'])
    toffsets = to_array(signals['ECG_T_Offsets'])

    RRinterval = calculate_consecutive_diff(rpeaks)
    PPinterval = calculate_consecutive_diff(ppeaks)

    Pduration = calculate_diff(ponsets, poffsets)
    PRsegment = calculate_diff(poffsets, qpeaks)
    PRinterval = calculate_diff(ponsets, qpeaks)
    PRsegment = calculate_diff(poffsets, qpeaks)
    QRScomplex = calculate_diff(qpeaks, speaks)
    QTinterval = calculate_diff(qpeaks, toffsets)
    STsegment = calculate_diff(speaks, tonsets)
    STTsegment = calculate_diff(speaks, toffsets)
    TPinterval = calculate_diff(toffsets, ponsets)
    
    return heart_rate, RRinterval, PPinterval, Pduration, PRsegment, PRinterval, \
        PRsegment, QRScomplex, QTinterval, STsegment, STTsegment, TPinterval

def get_values(template, r_peaks, peaks):
    result = []
    for i in range(len(peaks)):
        result.append(template[i][peaks[i] - r_peaks[i] + 60])
    return np.array(result)

def get_ecg_values(signal):
    result = ecg.ecg(signal, sampling_rate=sampling_rate, show=False)
    template = result['templates']

    p_peaks, p_start, p_end = getPPositions(result)

    q_peaks, q_start = getQPositions(result)
    for i in range(len(q_start)):
        if q_start[i] == p_peaks[i]:
            q_start[i] = int(p_end[i] + abs(q_peaks[i] - p_end[i]) / 2)

    r_peaks = result['rpeaks'].tolist()

    s_peaks, s_end = getSPositions(result)
    
    t_peaks, t_start, t_end = getTPositions(result)
    
    beats = fft.fft(template)
    heart_rate = sampling_rate * (60.0 / np.diff(result['rpeaks']))
    heart_rate = np.append(heart_rate, heart_rate[-1]).reshape(-1, 1)

    # They are of length = # heart beats - 1 !!!
    RRinterval = calculate_consecutive_diff(r_peaks)
    PPinterval = calculate_consecutive_diff(p_peaks)
    TPinterval = calculate_diff(t_end[:-1], p_start[1:])

    p_end = np.array(p_end)
    q_start = np.array(q_start)

    Pduration = p_end - p_start # calculate_diff(p_start, p_end)
    PRsegment = q_start - p_end # calculate_diff(p_end, q_start)
    PRinterval = q_start - p_start # calculate_diff(p_start, q_start)
    QRScomplex = calculate_diff(q_start, s_end)
    QTinterval = calculate_diff(q_start, t_end)
    STsegment = calculate_diff(s_end, t_start)
    STTsegment = calculate_diff(s_end, t_end)

    p_values = get_values(template, r_peaks, p_peaks)
    q_values = get_values(template, r_peaks, q_peaks)
    r_values = get_values(template, r_peaks, r_peaks)
    s_values = get_values(template, r_peaks, s_peaks)
    t_values = get_values(template, r_peaks, t_peaks)

    PQ_diff = calculate_diff(p_peaks, q_peaks)
    PR_diff = calculate_diff(p_peaks, r_peaks)
    PS_diff = calculate_diff(p_peaks, s_peaks)
    PT_diff = calculate_diff(p_peaks, t_peaks)
    QR_diff = calculate_diff(q_peaks, r_peaks)
    QS_diff = calculate_diff(q_peaks, s_peaks)
    QT_diff = calculate_diff(q_peaks, t_peaks)
    RS_diff = calculate_diff(r_peaks, s_peaks)
    RT_diff = calculate_diff(r_peaks, t_peaks)
    ST_diff = calculate_diff(s_peaks, t_peaks)

    return heart_rate, np.real(beats), np.imag(beats), \
        RRinterval, PPinterval, Pduration, PRsegment, PRinterval, QRScomplex, QTinterval, STsegment, STTsegment, TPinterval, \
        p_values, q_values, r_values, s_values, t_values, \
        PQ_diff, PR_diff, PS_diff, PT_diff, QR_diff, QS_diff, QT_diff, RS_diff, RT_diff, ST_diff
    #return p_start, p_peaks, p_end, q_start, q_peaks, r_peaks, s_peaks, s_end, t_start, t_peaks, t_end

def get_features(signal):
    features = []

    ecg_values_list = get_ecg_values(signal)
    for ecg_values in ecg_values_list:
        features += get_array_features(ecg_values)

    features += get_wavelet_features(signal)

    return features

def get_dataset_features(data):
    list_features = []
    for signal in data:
        list_features.append(get_features(signal))
    return list_features

In [15]:
X_train_extracted = get_dataset_features(X_train)

KeyboardInterrupt: 

In [ ]:
X_val_extracted = get_dataset_features(X_val)

# Classification

### Classifier definition

In [8]:
#cls = GradientBoostingClassifier(n_estimators=100, verbose=1, random_state=random_state)
cls = XGBClassifier(seed=random_state) #(n_estimators=100, gamma=1, reg_alpha=3, reg_lambda=0, max_depth=10, min_child_weight=0, colsample_bytree=0.85, seed=random_state)

## Classifier fit and evaluation

In [ ]:
cls.fit(X_train_extracted, y_train.T.ravel())

y_train_pred = cls.predict(X_train_extracted)
y_val_pred = cls.predict(X_val_extracted)

In [ ]:
train_score = f1_score(y_train, y_train_pred, average='micro')
val_score = f1_score(y_val, y_val_pred, average='micro')

print(train_score, val_score)

# Final classification

In [9]:
X_train_full_extracted = get_dataset_features(X_train_full)
X_test_extracted = get_dataset_features(X_test)

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC
svc = SVC(random_state=random_state)
svc_model = Pipeline([ ('scaler', StandardScaler()), ('svc', svc) ])

from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(random_state=random_state)
etc_model = Pipeline([ ('scaler', StandardScaler()), ('etc', etc) ])

from sklearn.ensemble import BaggingClassifier
bc = BaggingClassifier(estimator=svc_model, random_state=random_state)
bc_model = Pipeline([ ('scaler', StandardScaler()), ('bc', bc) ])

from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(estimator=svc_model, random_state=random_state)
abc_model = Pipeline([ ('scaler', StandardScaler()), ('abc', abc) ])

estimators = [
    ('xgbc', cls),
    ('svc', svc_model),
    #('gb', gb_model),
    ('etc', etc_model),
    #('gpr', gpr_model),
    ('bc', bc_model),
    ('abc', abc_model)
]

final_pipeline = Pipeline([ ('model', LogisticRegression()) ])
classifier = StackingClassifier(estimators, final_pipeline, n_jobs=n_cores)

In [14]:
y_test_pred = classifier.fit(X_train_full_extracted, y_train_full.T.ravel()).predict(X_test_extracted)

/Users/alrusso/AML-fall2023/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
# Just XGBClassifier
# y_test_pred = cls.fit(X_train_full_extracted, y_train_full.T.ravel()).predict(X_test_extracted)

### Writing results

In [ ]:
table = pd.DataFrame({'id': np.arange(0, y_test_pred.shape[0]), 'y': y_test_pred.flatten()})
table.to_csv(data_directory + 'y_test_pred.csv', index=False)